## SETI end-to-end narrowband drifting tone detection over multiple h5 spectrogram files
This notebook sequences through multiple h5 files and looks for drifting narrowband tones.  

This is a general-purpose notebook for real observations such as Voyager or commensal pointings.

For each test file, notebook "01_seti_end_to_end.ipynb" is called which performs the following:

1) Inputs a single h5 spectrum file.  The fine FFT size and integration factor are already determined.
2) Runs TurboSETI, seticore or bliss and compiles a list of detections and compute time (wall clock)

Note that these sets of files can be done with various branches of seticore or seticore2 and detection SNR or timing comparisons
can be made between branches over multiple runs of this notebook.  For each run of this notebook, 
the desired seticore branch needs to be checked out and compiled, and 
the string "test_case" needs to be set in 01_seti_end_to_end.ipynb to indicate the branch.

The h5 files can be run from local storage ($SGDIR) or they be run directly from bulk storage without copying locally. 

Plots and .dat files (indicating "hits") are generated in each "01_seti_end_to_end.ipynb" call.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
import os
import subprocess
import glob


sg_dir0 = os.getenv('SGDIR') + '/'
if not os.path.isdir(sg_dir0[0:-1]):
    print(f"Warning: ")

parameters_are_undefined = False     # use parameter defaults settings in 01_seti_end_to_end.ipynb

# app_to_run = 'turbo-10'         # turbo-<search_z_threshold ('snr')> 
app_to_run = 'seticore-10'
# app_to_run = 'bliss-10-8'     # bliss-use for setigen raw files - <search_z_threshold>-<pfb_order>
# app_to_run = 'bliss-10-12'      # bliss- 12th order PFB (GBT)
# app_to_run = 'bliss-10-101'     # use special file 'GBT_spliced_PFB_response.f32'
# app_to_run = 'Bliss_C+-4_dat-10'     # plot dat file for bliss runs 
# app_to_run = 'Bliss_C0-11_dat-10'     # plot dat file for bliss runs 
        
display_figs01 = False
plot_all_dets = True
verbose = True
plot_sg_zoom = False
plot_psd_zoom = False
plot_multi_psd_zoom = False
plot_sg_wb = False
plot_psd_wb = False
plot_channel_idx_dets = False
plot_det_histogram = False

plot_sg_psd_zoom = False
max_n_BB_zoom_plots = 20
max_n_NB_zoom_plots = 20
max_n_multi_zoom_plots = 12
zoom_NB_bw_MHz = .030   # +/- 15 KHz
zoom_min_max_freq_MHz = []
zoom_min_max_drift_rate = []
beep_when_h5_complete = False

run_turbo = False
fb_ext = '.h5'

h5_backup_subdir = ''
base_h5_name = '' 

search_max_drift = 10    # Hz/sec
search_min_drift = -10   # ignored except for plotting
min_abs_drift_Hz_sec = 0.01
dc_blank = True

base_h5_name_list = []
sg_dir = sg_dir0

if (0):
    base_h5_name_list = [
        'blc73_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
        'blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
        'uniform_blc73_guppi_58832_16209_MESSIER031_0057.rawspec.0000.h5']
    fig_dir = './mess_plots/'  
elif (1):
    h5_file_spec = ['blc*16209_MESSIER031*.h5','uniform*.h5','blc*HIP*.h5','blc*TIC*.h5','gbt-chirp80*.h5'];
    fig_dir = './full_plots/'  
    plot_multi_psd_zoom = True
elif (1):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/HIP_TIC_C_59239/'
    h5_file_spec = ['*.h5'];
    fig_dir = './HIP_TIC_C_59239_plots/'  
elif (0):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/HIP_TIC_L_59756/'
    h5_file_spec = ['*.h5'];
    fig_dir = './HIP_TIC_L_59756_plots/'  
elif (1):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/HIP_TIC_X_59189/'
    h5_file_spec = ['*.h5'];
    fig_dir = './HIP_TIC_X_59189_plots/'  
elif (0):
    # examples of Bliss drift rates at rails
    sg_dir = '/mnt/d/data_files/seti_benchmarking/HIP_TIC_C_59239/'
    h5_file_spec = ['blc10*.h5','blc20*.h5','blc30*.h5','blc35*.h5','blc37*.h5'];
    plot_channel_idx_dets = True
    plot_det_histogram = True
    fig_dir = './HIP_TIC_C_59239_channel_plots/'  
elif (0):
    h5_file_spec = 'gbt-chirp80*.h5';
    fig_dir = './chirp_plots/'  
elif (1):
    # Nsti=16
    sg_dir = '/mnt/d/data_files/seti_benchmarking/KEPLER-160/'
    h5_file_spec = ['*.h5']
    fig_dir = './KEPLER_plots/'  
elif (0):
    # part of full_plots
    sg_dir = '/mnt/d/data_files/seti_benchmarking/MESS_58832/'
    h5_file_spec = ['*.h5']
    fig_dir = './mess_plots/'  
elif (0):
    h5_file_spec = ['blc*HIP*.h5','blc*TIC*.h5'];
    fig_dir = './HIP_TIC_plots/'  
elif (0):
    # h5_file_spec = '*.h5';
    h5_file_spec = 'blc*16209_MESSIER031*.h5';
    # h5_file_spec = '*TIC159107668_0016*.h5';
    fig_dir = './plots/'  
elif (0):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/mlc/'
    h5_file_spec = '*HIP*.h5';
    # base_h5_name_list = [
    #     'spliced_blc0001020304050607_guppi_57517_08789_HIP54677_0009.gpuspec.0000.h5',
    #     'spliced_blc0001020304050607_guppi_57517_09209_HIP53759_0010.gpuspec.0000.h5']
    fig_dir = './mlc_plots/'  
elif (0):
    # problems reading file
    sg_dir = '/mnt/d/data_files/seti_benchmarking/lband_rfi_single_node/'
    h5_file_spec = '*.fil';
    fig_dir = './lband_rfi_plots/'  
elif (0):
    # problems reading file
    sg_dir = '/mnt/d/data_files/seti_benchmarking/lband_clean_single_node/'
    h5_file_spec = '*.fil';
    fig_dir = './lband_clean_plots/'  
elif (0):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/Sample_Datasets_ASP/Catalogued_Signals/'
    h5_file_spec = '*.h5';
    fig_dir = './ASP_plots/'  
    plot_sg_psd_zoom = True
elif (1):
    sg_dir = '/mnt/d/data_files/seti_benchmarking/Sample_Datasets_ASP/Horseshoe_Artifact/'
    h5_file_spec = '*.h5';
    fig_dir = './ASP_plots/'  
    plot_sg_psd_zoom = False
elif (0):
    # problems reading file
    sg_dir = '/mnt/d/data_files/seti_benchmarking/cosmic_h5/'
    h5_file_spec = '*.h5';
    fig_dir = './cosmic_plots/'  
    plot_sg_psd_zoom = True
elif (0):
    base_h5_name_list = [
        'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-45.81sec-1024K-Nsti-16.rawspec.0000.h5',
        'gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-10.0-Hzsec-366.50sec-1024K-Nsti-16.rawspec.0000.h5']
    zoom_bw_MHz = .012  # +/- 6 KHz
    fig_dir = './gbt_chirp_plots/'
elif (0):
    base_h5_name_list = [
        'blc23_guppi_59046_80036_DIAG_VOYAGER-1_0011-1024K-Nsti-16.rawspec.0000.h5',
        'blc43_guppi_59103_01984_DIAG_KEPLER-160_0010-1024K-Nsti-16.rawspec.0000.h5',
        'blc45_guppi_59103_01984_DIAG_KEPLER-160_0010-1024K-Nsti-16.rawspec.0000.h5']
    fig_dir = './plots/'

# if file spec entered, obtain list of files

if (len(base_h5_name_list)==0):
    if not isinstance(h5_file_spec,list):
        for file in glob.glob(sg_dir+h5_file_spec):
            base_h5_name_list.append(os.path.basename(file))
    else:
        for i_list in range(len(h5_file_spec)):
            for file in glob.glob(sg_dir+h5_file_spec[i_list]):
                base_h5_name_list.append(os.path.basename(file))

n_h5_file = len(base_h5_name_list)

print(f'SG directory={sg_dir}')
print(f'base_h5_name_list= ({n_h5_file} files)')
for i_h5 in range(n_h5_file):
    print(f'{base_h5_name_list[i_h5]}')
             
print(f'Figures will appear in in {fig_dir}')

SG directory=/home/kmh4005/data_files/sg_det_files/
base_h5_name_list= (28 files)
blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blc76_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blc74_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blc73_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
uniform_blc73_guppi_58832_16209_MESSIER031_0057.rawspec.0000.h5
blc45_guppi_59103_03394_DIAG_HIP95631_0015-1024K-Nsti-16.rawspec.0000.h5
blc41_guppi_59103_03394_DIAG_HIP95631_0015-1024K-Nsti-16.rawspec.0000.h5
blc1_guppi_57388_HIP113357_0010-1009K-Nsti-16.rawspec.0000.h5
blc47_guppi_59103_03394_DIAG_HIP95631_0015-1024K-Nsti-16.rawspec.0000.h5
blc00_guppi_59239_37260_HIP50422_0024.rawspec.0000.h5
blc20_guppi_59239_37260_HIP50422_0024.rawspec.0000.h5
blc43_guppi_59103_03394_DIAG_HIP95631_0015-1024K-Nsti-16.rawspec.0000.h5
blc40_guppi_59103_03394_DIAG_HIP95631_0015-1024K-Nsti-16

#### Run end-to-end test over parameter sweeps

In [2]:

for i_h5 in range(n_h5_file):
    base_h5_name = base_h5_name_list[i_h5]
    print(f'\n\nh5 file {i_h5+1} of {n_h5_file}: {base_h5_name}\n\n')

    try:      
        %run ./10_seti_h5_find_hits.ipynb
        plt.close('all')
    except:
        print(f'\n\nError Occurred for h5 file {base_h5_name}, skip\n\n')

    

            



h5 file 1 of 28: blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5


min_abs_drift_Hz_sec=0.01
search_app_name='seticore2', search_app_string='Seticore2', branch='sc4', test_case='sc4-Box40-10', search_z_threshold=10
base_h5_name = blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5
blimpy.io.base_reader WARNING  Selection size is 4.00 GB, which exceeds the memory usage limit of 2.7466049194335938 GB. Keeping data on disk.
base_h5_name = '/home/kmh4005/data_files/sg_det_files/blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5'
h5 file found
h5 File size =   3112 MB, n_h5_files = 1

GBT, 2019-12-15T04:30:09.000, mjd_day=58832, MESSIER031, ra_deg = 10.685, dec_deg = 41.268
ctr_freq_MHz = 1970.215, obs_bw_MHz = -187.5, chan_bw = -2929687.500, f_min_MHz = 1876.465, f_max_MHz = 2063.965

t_obs = 292.058, n_coarse_channels = 64, n_bits = 32, chan_bw = -2929687.500
fs_coarse = 2929687.500, fine_fft_size = 1048576, n_sti = 51, n_lti = 16, n_avg = 816
t_coarse = 3.413e-07, t_fi

mkdir: cannot create directory ‘./full_plots/sc4-Box40-10’: File exists
dedoppler elapsed time: 71s



Seticore search: 70.99 sec 1.11 sec/coarse channel
df_dash = 0.002861 MHz
No setigen data available
0 Inserted chirps
f_start =  []
BB: range(0, 2)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[2062.500000265625]
NB: range(0, 20)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[]
i_det+1=7, dc_freq_MHz=[]
i_det+1=8, dc_freq_MHz=[]
i_det+1=9, dc_freq_MHz=[]
i_det+1=10, dc_freq_MHz=[]
i_det+1=11, dc_freq_MHz=[]
i_det+1=12, dc_freq_MHz=[]
i_det+1=13, dc_freq_MHz=[]
i_det+1=14, dc_freq_MHz=[]
i_det+1=15, dc_freq_MHz=[]
i_det+1=16, dc_freq_MHz=[1968.749998632565]
i_det+1=17, dc_freq_MHz=[]
i_det+1=18, dc_freq_MHz=[]
i_det+1=19, dc_freq_MHz=[]
i_det+1=20, dc_freq_MHz=[]
i_det+1=21, dc_freq_MHz=[]
i_det+1=22, dc_freq_MHz=[]
i_det+1=23, dc_freq_MHz=[]
i_det+1=24, dc_freq_MHz=[]
i_det+1=25, dc_freq_MHz=[]
i_det+1=26, dc_freq_MHz=[]
i_det+1=27, dc_freq_MHz=[]
i_det+1=28, dc_freq_MHz=[]
i

mkdir: cannot create directory ‘./full_plots/sc4-Box40-10’: File exists
dedoppler elapsed time: 73s



Seticore search: 73.05 sec 1.14 sec/coarse channel
df_dash = 0.002861 MHz
No setigen data available
0 Inserted chirps
f_start =  []
BB: range(0, 20)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[1130.8593136132813]
i_det+1=7, dc_freq_MHz=[]
i_det+1=8, dc_freq_MHz=[]
i_det+1=9, dc_freq_MHz=[]
i_det+1=10, dc_freq_MHz=[1133.7890015117187]
i_det+1=11, dc_freq_MHz=[]
i_det+1=12, dc_freq_MHz=[]
i_det+1=13, dc_freq_MHz=[]
i_det+1=14, dc_freq_MHz=[1142.57812525]
i_det+1=15, dc_freq_MHz=[1145.50781225]
i_det+1=16, dc_freq_MHz=[1148.43750025]
i_det+1=17, dc_freq_MHz=[1151.367187328125]
i_det+1=18, dc_freq_MHz=[]
i_det+1=19, dc_freq_MHz=[]
i_det+1=20, dc_freq_MHz=[]
i_det+1=21, dc_freq_MHz=[1160.1561894335937]
i_det+1=22, dc_freq_MHz=[1163.0859374921874]
i_det+1=23, dc_freq_MHz=[]
i_det+1=24, dc_freq_MHz=[1168.94531225]
i_det+1=25, dc_freq_MHz=[1171.87500025]
i_det+1=26, dc_freq_MHz=[1174.

...


Seticore search: 7.04 sec 0.11 sec/coarse channel
df_dash = 0.002861 MHz
No setigen data available
0 Inserted chirps
f_start =  []
BB: range(0, 6)
i_det+1=1, dc_freq_MHz=[]
blimpy.io.base_reader WARNING  Setting f_start = 1313.964844, since f_start not given or not valid.
blimpy.io.base_reader WARNING  f_start=1313.9148440000001, f_stop=1314.243726, t_start=0, t_stop=4, init=True
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[1315.4296263476563]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[]
NB: range(0, 12)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[]
i_det+1=7, dc_freq_MHz=[]
i_det+1=8, dc_freq_MHz=[]
i_det+1=9, dc_freq_MHz=[]
i_det+1=10, dc_freq_MHz=[1499.999999642738]
i_det+1=11, dc_freq_MHz=[1500.000000357262]
i_det+1=12, dc_freq_MHz=[]
seti_h5_find_hits completed, 15.84 sec



h5 file 27 of 28: gbt-chirp80-0.0020V-6002.20-6003.80-m10.0-

mkdir: cannot create directory ‘./full_plots/sc4-Box40-10’: File exists
dedoppler elapsed time: 7s



Seticore search: 6.94 sec 1.73 sec/coarse channel
df_dash = 0.002861 MHz
No setigen data available
0 Inserted chirps
f_start =  []
NB: range(0, 20)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[]
i_det+1=7, dc_freq_MHz=[]
i_det+1=8, dc_freq_MHz=[]
i_det+1=9, dc_freq_MHz=[]
i_det+1=10, dc_freq_MHz=[]
i_det+1=11, dc_freq_MHz=[]
i_det+1=12, dc_freq_MHz=[]
i_det+1=13, dc_freq_MHz=[]
i_det+1=14, dc_freq_MHz=[]
i_det+1=15, dc_freq_MHz=[]
i_det+1=16, dc_freq_MHz=[]
i_det+1=17, dc_freq_MHz=[]
i_det+1=18, dc_freq_MHz=[]
i_det+1=19, dc_freq_MHz=[]
i_det+1=20, dc_freq_MHz=[]
i_det+1=21, dc_freq_MHz=[]
i_det+1=22, dc_freq_MHz=[]
i_det+1=23, dc_freq_MHz=[]
i_det+1=24, dc_freq_MHz=[]
i_det+1=25, dc_freq_MHz=[]
i_det+1=26, dc_freq_MHz=[]
i_det+1=27, dc_freq_MHz=[]
i_det+1=28, dc_freq_MHz=[]
i_det+1=29, dc_freq_MHz=[]
i_det+1=30, dc_freq_MHz=[]
i_det+1=31, dc_freq_MHz=[]
i_det+1=32, dc_freq_MHz

mkdir: cannot create directory ‘./full_plots/sc4-Box40-10’: File exists
dedoppler elapsed time: 1s



Seticore search: 1.23 sec 0.31 sec/coarse channel
df_dash = 0.002861 MHz
No setigen data available
0 Inserted chirps
f_start =  []
NB: range(0, 20)
i_det+1=1, dc_freq_MHz=[]
i_det+1=2, dc_freq_MHz=[]
i_det+1=3, dc_freq_MHz=[]
i_det+1=4, dc_freq_MHz=[]
i_det+1=5, dc_freq_MHz=[]
i_det+1=6, dc_freq_MHz=[]
i_det+1=7, dc_freq_MHz=[]
i_det+1=8, dc_freq_MHz=[]
i_det+1=9, dc_freq_MHz=[]
i_det+1=10, dc_freq_MHz=[]
i_det+1=11, dc_freq_MHz=[]
i_det+1=12, dc_freq_MHz=[]
i_det+1=13, dc_freq_MHz=[]
i_det+1=14, dc_freq_MHz=[]
i_det+1=15, dc_freq_MHz=[]
i_det+1=16, dc_freq_MHz=[]
i_det+1=17, dc_freq_MHz=[]
i_det+1=18, dc_freq_MHz=[]
i_det+1=19, dc_freq_MHz=[]
i_det+1=20, dc_freq_MHz=[]
i_det+1=21, dc_freq_MHz=[]
i_det+1=22, dc_freq_MHz=[]
i_det+1=23, dc_freq_MHz=[]
i_det+1=24, dc_freq_MHz=[]
i_det+1=25, dc_freq_MHz=[]
i_det+1=26, dc_freq_MHz=[]
i_det+1=27, dc_freq_MHz=[]
i_det+1=28, dc_freq_MHz=[]
i_det+1=29, dc_freq_MHz=[]
i_det+1=30, dc_freq_MHz=[]
i_det+1=31, dc_freq_MHz=[]
i_det+1=32, dc_freq_MHz

In [3]:
import os

print('All Runs Done')

# Beep 2 times
for i_beep in range(2):
    try:
        # Beep in WSL
        if os.system("powershell.exe '[console]::beep(261.6,700)'") !=0:
            raise Exception('powershell.exe not found')
    except:
        # linux, probably doesn't work
        print('Beep!')
        os.system("echo -ne '\a'")

All Runs Done
